# Workflow
In this example, we will use McStas 3 simulation file.

## Build Pipeline (Collect Parameters and Providers)
Import the providers from ``load_mcstas_nexus`` to use the ``McStas`` simulation data workflow. <br>
``MaximumProbability`` can be manually provided to derive more realistic number of events. <br>
It is because ``weights`` are given as probability, not number of events in a McStas file. <br>

In [ ]:
from ess.nmx.mcstas import McStasWorkflow
from ess.nmx.data import small_mcstas_3_sample

from ess.nmx.types import *
from ess.nmx.reduction import NMXReducedData, merge_panels
from ess.nmx.nexus import export_as_nexus

base_wf = McStasWorkflow()  # Instantiate the base workflow
base_wf[FilePath] = small_mcstas_3_sample()  # Replace with the path to your own file
base_wf[MaximumProbability] = 10_000
base_wf[TimeBinSteps] = 50

### All Types and Providers of the Workflow

In [ ]:
base_wf

### Graph of the Workflow.

In [ ]:
base_wf.visualize(NMXReducedData, graph_attr={"rankdir": "LR"})

### Detector Index Mapping

We want to reduce all three panels, so we map the relevant part of the workflow over a list of the three panels:

In [ ]:
# DetectorIndex selects what detector panels to include in the run
# in this case we select all three panels.
detector_panel_ids = {DetectorIndex: sc.arange('panel', 3, unit=None)}
pipeline = base_wf.map(detector_panel_ids)

### Visualize Mapped Graph

In [ ]:
import sciline as sl

pipeline.visualize(
    sl.get_mapped_node_names(pipeline, NMXReducedData),
    graph_attr={"rankdir": "LR"},
    compact=True,
)

### Reduce Mapped Results

In [ ]:
from typing import NewType

MergedNMXReducedData = NewType('MergedNMXReducedData', sc.DataGroup)
graph = pipeline.reduce(func=merge_panels, name=MergedNMXReducedData)
graph.get(MergedNMXReducedData).visualize(compact=True, graph_attr={"rankdir": "LR"})

## Compute Desired Types

In [ ]:
binned_dg = graph.compute(MergedNMXReducedData)
binned_dg

## Export Results

``NMXReducedData`` object has a method to export the data into nexus or h5 file.

You can save the result as ``test.nxs``, for example:


In [ ]:
export_as_nexus(binned_dg, "test.nxs")

### Merge Intermediate Mapped Nodes

In [ ]:
from ess.nmx.reduction import NMXData

dg = merge_panels(
    *pipeline.compute(sl.get_mapped_node_names(pipeline, NMXData)).values()
)  # We need ``NMXData`` to use instrument view
dg

## Instrument View

Pixel positions are not used for later steps,
but it is included in the coordinates for instrument view.

All pixel positions are relative to the sample position,
therefore the sample is at (0, 0, 0).

**It might be very slow or not work in the ``VS Code`` jupyter notebook editor.**

In [ ]:
import scippneutron as scn

da = binned_dg['counts'].sum('t')
da.coords["position"] = binned_dg["position"]
# Plot one out of 100 pixels to reduce size of docs output
view = scn.instrument_view(da["id", ::100], pixel_size=0.0075)
view